### Reading in data

The data are CSV files. In Spark, these can be read using textFile and splitting rows on commas.

In [1]:
%%writefile spark_analysis.py
import matplotlib
matplotlib.use('agg')
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--bucket", help="bucket for input and output")
args = parser.parse_args()
BUCKET = args.bucket

Writing spark_analysis.py


In [2]:
%%writefile -a spark_analysis.py
from pyspark.sql import SparkSession, SQLContext, Row
gcs_bucket='sincere-pen-389114'
spark = SparkSession.builder.appName("kdd").getOrCreate()
sc = spark.sparkContext
data_file = "gs://"+gcs_bucket+"//kddcup.data_10_percent.gz"
raw_rdd = sc.textFile(data_file).cache()
raw_rdd.take(5)

Appending to spark_analysis.py


In [3]:
%%writefile -a spark_analysis.py
csv_rdd = raw_rdd.map(lambda row: row.split(","))
parsed_rdd = csv_rdd.map(lambda r: Row(
    duration=int(r[0]), 
    protocol_type=r[1],
    service=r[2],
    flag=r[3],
    src_bytes=int(r[4]),
    dst_bytes=int(r[5]),
    wrong_fragment=int(r[7]),
    urgent=int(r[8]),
    hot=int(r[9]),
    num_failed_logins=int(r[10]),
    num_compromised=int(r[12]),
    su_attempted=r[14],
    num_root=int(r[15]),
    num_file_creations=int(r[16]),
    label=r[-1]
    )
)
parsed_rdd.take(5)

Appending to spark_analysis.py


### Spark analysis

One way to analyze data in Spark is to call methods on a dataframe.

In [4]:
%%writefile -a spark_analysis.py
sqlContext = SQLContext(sc)
df = sqlContext.createDataFrame(parsed_rdd)
connections_by_protocol = df.groupBy('protocol_type').count().orderBy('count', ascending=False)
connections_by_protocol.show()

Appending to spark_analysis.py


Another way is to use Spark SQL

In [5]:
%%writefile -a spark_analysis.py
df.registerTempTable("connections")
attack_stats = sqlContext.sql("""
                           SELECT 
                             protocol_type, 
                             CASE label
                               WHEN 'normal.' THEN 'no attack'
                               ELSE 'attack'
                             END AS state,
                             COUNT(*) as total_freq,
                             ROUND(AVG(src_bytes), 2) as mean_src_bytes,
                             ROUND(AVG(dst_bytes), 2) as mean_dst_bytes,
                             ROUND(AVG(duration), 2) as mean_duration,
                             SUM(num_failed_logins) as total_failed_logins,
                             SUM(num_compromised) as total_compromised,
                             SUM(num_file_creations) as total_file_creations,
                             SUM(su_attempted) as total_root_attempts,
                             SUM(num_root) as total_root_acceses
                           FROM connections
                           GROUP BY protocol_type, state
                           ORDER BY 3 DESC
                           """)
attack_stats.show()

Appending to spark_analysis.py


In [6]:
%%writefile -a spark_analysis.py
ax = attack_stats.toPandas().plot.bar(x='protocol_type', subplots=True, figsize=(10,25))

Appending to spark_analysis.py


In [7]:
%%writefile -a spark_analysis.py
ax[0].get_figure().savefig('report.png');

Appending to spark_analysis.py


In [8]:
%%writefile -a spark_analysis.py
import google.cloud.storage as gcs
bucket = gcs.Client().get_bucket(BUCKET)
for blob in bucket.list_blobs(prefix='sparktodp/'):
    blob.delete()
bucket.blob('sparktodp/report.png').upload_from_filename('report.png')

Appending to spark_analysis.py


In [9]:
%%writefile -a spark_analysis.py
connections_by_protocol.write.format("csv").mode("overwrite").save(
    "gs://{}/sparktodp/connections_by_protocol".format(BUCKET))

Appending to spark_analysis.py


In [10]:
BUCKET_list = !gcloud info --format='value(config.project)'
BUCKET=BUCKET_list[0]
print('Writing to {}'.format(BUCKET))
!/opt/conda/miniconda3/bin/python spark_analysis.py --bucket=$BUCKET

Writing to sincere-pen-389114
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/26 05:45:10 INFO SparkEnv: Registering MapOutputTracker
23/07/26 05:45:10 INFO SparkEnv: Registering BlockManagerMaster
23/07/26 05:45:10 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/07/26 05:45:10 INFO SparkEnv: Registering OutputCommitCoordinator
23/07/26 05:45:20 WARN GhfsStorageStatistics: Detected potential high latency for operation op_get_file_status. latencyMs=452; previousMaxLatencyMs=0; operationCount=1; context=gs://dataproc-temp-us-central1-387021963560-qcsdihcz/45346b86-8b3d-4e0d-8fa1-b8c6db524c56/spark-job-history
23/07/26 05:45:20 WARN GhfsStorageStatistics: Detected potential high latency for operation op_mkdirs. latencyMs=206; previousMaxLatencyMs=0; operationCount=1; context=gs://dataproc-temp-us-central1-387021963560-qcsdihcz/45346b86-8b3d-4e0d-8fa1-b8c6db524c56/spark-job-history
23/07/26 05

In [11]:
!gcloud storage ls gs://$BUCKET/sparktodp/**

gs://sincere-pen-389114/sparktodp/connections_by_protocol/
gs://sincere-pen-389114/sparktodp/connections_by_protocol/_SUCCESS
gs://sincere-pen-389114/sparktodp/connections_by_protocol/part-00000-ce2a0ea9-a46f-4cff-843f-20e42d4c1cde-c000.csv
gs://sincere-pen-389114/sparktodp/report.png


In [12]:
!gcloud storage cp spark_analysis.py gs://$BUCKET/sparktodp/spark_analysis.py

Copying file://spark_analysis.py to gs://sincere-pen-389114/sparktodp/spark_analysis.py
  Completed files 1/1 | 2.8kiB/2.8kiB                                          


Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.